# Análise Bivariada e Multivariada

Nesta seção, exploraremos as relações entre pares de variáveis. Isso é fundamental para visualizar correlações e levantar hipóteses sobre dependências causais. É lamentável que nosso conjunto de dados sobre colisões com pássaros capture apenas incidentes em que ocorreram colisões com pássaros, e não também voos em que as colisões foram evitadas. Embora pudéssemos buscar dados externos para esses voos sem incidentes e adicioná-los a este conjunto de dados, teremos que nos virar sem eles. Ainda encontraremos algumas descobertas interessantes.

Vamos trazer nosso conjunto de dados limpo novamente e fazer as conversões apropriadas.

In [ ]:
import pandas as pd 

url = r"https://github.com/thomasnield/anaconda_python_eda/raw/public/birdstrike_section2.csv"
df = pd.read_csv(url, index_col='INDEX_NR', parse_dates=["INCIDENT_DATE"])

# Transforme PHASE_OF_FLIGHT em uma categoria
phase_of_flt = pd.CategoricalDtype(categories=['Parked', 'Taxi','Take-off Run', 'Approach', 'Departure', 'Climb', 'En Route',
                                               'Descent', 'Landing Roll', 'Arrival', 'Local'])

df["PHASE_OF_FLIGHT"] = df["PHASE_OF_FLIGHT"].astype(phase_of_flt)

# Transforme TIME em tipo timedelta
df["TIME"] = pd.to_timedelta(df["TIME"])

## Comparando Altura com Velocidade

Analisamos `HEIGHT` e `SPEED` extensivamente na última seção, mas analisamos cada uma separadamente. Vamos ver se há alguma relação entre as duas e algo interessante. O ponto de partida mais informativo para isso é um gráfico de dispersão.

In [ ]:
df.plot.scatter(x="HEIGHT",y="SPEED")

Nossa, já tem muita coisa para analisar aqui. Parece haver uma correlação linear entre `SPEED` e `HEIGHT`. Podemos quantificar essa correlação usando a [correlação de Pearson](https://youtu.be/rijqfllOq6g?si=Ls9OkGqCO6AxnWN2), onde um valor próximo a 1 é uma forte correlação positiva, próximo a 0 é nenhuma correlação e próximo a -1 é uma forte correlação negativa.

In [ ]:
df[["HEIGHT","SPEED"]].corr(method='pearson')

Assim, obtemos uma correlação de `0,681334`, que é uma correlação positiva razoável. Isso faz sentido porque a maioria dos voos sobe para altitudes mais altas para obter mais velocidade, que em seu pico é chamada de altitude de cruzeiro.

Mas e aqueles 3 pontos de dados perto de 0 pés, mas que estavam incrivelmente rápidos, entre 600 e 1200 nós! 1200 nós é cerca de Mach 1,8, então será que o Maverick estava sobrevoando a torre de novo?

![](https://y.yarn.co/18f66e28-0f40-4db7-b05f-3529759e9708_text.gif)

*Cortesia: Paramount Pictures*

> De fato, houve uma cena de colisão com pássaros no filme Top Gun: Maverick
>
> https://youtu.be/NCrsxRgcC38?si=mdeSiNNYs4iM9dgX

Vamos consultar esses três pontos de dados como valores atípicos.

In [ ]:
with pd.option_context('display.max_columns', None):
    display(df[df["SPEED"] > 600])

Ok... Tenho perguntas. Um Cessna 172 a 650 nós (a velocidade máxima é 163 nós)? E um Boeing 737-700 voando a 1150 nós a 1000 pés acima do solo (seu máximo é 473 nós)? E um Flexjet voando a 1250 nós a 100 pés acima do solo? Até mesmo o F-18 da Maverick mal consegue atingir Mach 1,8, ou 1200 nós. Esses pontos de dados devem ter sido registrados incorretamente, a menos que estejamos realmente perdendo algo. Devemos considerar removê-los se os usarmos para modelagem.

Vamos comparar separadamente essas duas variáveis ​​com `DISTANCE`.

In [ ]:
df.plot.scatter(x="DISTANCE",y="HEIGHT")

In [ ]:
df.plot.scatter(x="DISTANCE",y="SPEED")

Use um gráfico de pares para comparar cada variável com todas as outras simultaneamente.

In [ ]:
import seaborn as sns

sns.pairplot(df[["HEIGHT","SPEED","DISTANCE"]])

Agora, vamos analisar as correlações entre essas três variáveis. Vemos que `HEIGHT`, `SPEED` e `DISTANCE` têm correlações entre si. Há uma correlação especialmente forte entre `HEIGHT` e `DISTANCE` de `.834154`. Isso faz sentido porque um avião estará descendo conforme se aproxima do pouso em um aeroporto.

In [ ]:
corr_matrix = df[["HEIGHT","SPEED","DISTANCE"]].corr(method='pearson')
corr_matrix

Também podemos visualizar isso usando um mapa de calor.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True)

Poderíamos tentar trazer mais variáveis, ou até mesmo o dataframe inteiro. No entanto, como muitas dessas variáveis ​​são discretas e precisariam ser convertidas numericamente, não faremos isso aqui. Mas você pode explorar outras variáveis ​​como parte do exercício e usar seu conhecimento para encontrar mais correlações.

Para que usaríamos essas informações de correlação? Bem, por exemplo, se estivéssemos usando aprendizado de máquina como regressão logística para prever a probabilidade de colisões com pássaros, não iríamos querer incluir variáveis colineares que estejam redundantemente correlacionadas entre si. Isso acabaria dominando a variável causal subjacente no modelo. Provavelmente escolheríamos apenas uma dessas três variáveis ​​para fazer uma previsão, em vez de todas as três, como `HEIGHT`.

## Uma Breve Discussão sobre Variáveis ​​Preditoras

Se estamos preocupados com colisões com pássaros, é lamentável que tenhamos apenas dados autorrelatados de quando elas ocorreram. Para ver quais variáveis ​​podem ser usadas para prever colisões com pássaros, precisamos de dados de voos normais onde não ocorreram colisões com pássaros. Não é impossível obter esses dados, mas está além do escopo deste exercício. No entanto, ainda vale a pena discuti-los.

Por exemplo, é uma pena que não tenhamos registros de voos sem colisões com pássaros e que tivemos um alerta. Poderíamos ter usado a variável "WARNED", como mostrado abaixo, e feito alguns testes para verificar se os alertas tiveram algum efeito na prevenção de colisões com pássaros. Mas é difícil usar essa variável porque alertas bem-sucedidos que evitaram colisões com pássaros provavelmente não foram registrados.

In [ ]:
df["WARNED"].value_counts(dropna=False).plot.bar()

Se tivéssemos acesso aos horários de voos para dados não incidentes, bem como aos dados meteorológicos, também poderíamos verificar se `PRECIPITATION` aumentou o número per capita de incidentes quando ocorreu chuva. Mas sem esses dados, é difícil concluir qualquer coisa. Sim, mais colisões com pássaros acontecem sem chuva em nossos dados. Mas se a chuva não for frequente, isso criará menos registros. Isso não significa que a taxa de incidentes não será maior com chuva.

In [ ]:
df["PRECIPITATION"].fillna(value="", inplace=False).str.contains("Rain").value_counts().plot.bar()

Os dados de colisões com pássaros não capturam isso, mas o fato é que as colisões com pássaros aumentam APÓS a chuva, porque os pássaros gostam de banhar em poças d'água no solo. A má gestão do escoamento nas pistas pode, portanto, aumentar as colisões com pássaros. Isso apenas demonstra que seus dados capturam apenas uma parte limitada da verdade.

## Análise Multivariada

Tentar analisar relações entre três ou mais variáveis ​​pode ser desafiador de visualizar, mas existem maneiras se você fizer isso com cuidado. Neste exercício, vou avaliar se uma colisão com pássaros tem maior probabilidade de ocorrer no motor (seja um motor a jato, hélice ou rotor).

Em retrospectiva, percebi que omiti algumas colunas importantes onde uma colisão, dano ou ingestão ocorreram na seção 1. Mas este será um bom exercício para aprender a corrigir esse erro. Vamos trazer esses dados do arquivo Excel.

In [ ]:
import pandas as pd

engine_cols = ["INDEX_NR", 
"STR_ENG1",
"DAM_ENG1",
"ING_ENG1",
"STR_ENG2",
"DAM_ENG2",
"ING_ENG2",
"STR_ENG3",
"DAM_ENG3",
"ING_ENG3",
"STR_ENG4",
"DAM_ENG4",
"ING_ENG4",
"STR_PROP",
"DAM_PROP",
"STR_WING_ROT",
"DAM_WING_ROT"]

url = r"https://github.com/thomasnield/anaconda_python_eda/raw/public/bird_strike_faa.xlsx"
engine_df = pd.read_excel(url, index_col='INDEX_NR', usecols=engine_cols)
engine_df

Vamos consolidar todas essas colunas em uma, somando-as para cada linha respectiva. Em seguida, faremos disso um indicador booleano usando uma operação comparativa, verificando se quaisquer valores maiores que 0 são considerados `True` e, caso contrário, serão considerados `False`.

In [ ]:
engine_df_ind = engine_df.sum(axis=1) > 0 
engine_df_ind

Vamos realizar uma junção à esquerda em nosso dataframe, o que significa que manteremos apenas registros em nosso `df` de dados de 2015 em diante e incluiremos apenas dados `ENGINE_STRIKE_IND` que tenham índices em comum.

In [ ]:
df = df.join(engine_df_ind.rename("ENGINE_STRIKE_IND"), how='left')

with pd.option_context('display.max_columns', None):
    display(df)

Agora, quantas colisões desde 2015 foram relacionadas a motores? Descobrimos que são mais de 20% das colisões.

In [ ]:
df["ENGINE_STRIKE_IND"].value_counts()

Quando se trata de análise multivariável, podemos ficar realmente malucos e precisamos estruturar nossas hipóteses cuidadosamente. Afinal, tentar encontrar a relação entre mais de duas variáveis ​​pode ser bastante complicado, e é fácil se perder no meio do ruído.

Para nossos propósitos, digamos que estamos interessados ​​em entender como a massa, a altura e a velocidade da aeronave influenciam se uma colisão com pássaros está ou não relacionada ao motor em diferentes altitudes. Parece complicado, não é? Como disse o Temido Pirata Roberts em *A Princesa Prometida*: "Você realmente tem um intelecto estonteante."

> A coluna `AC_MASS` pode ser interpretada da seguinte forma: 1 = 2.250 kg ou menos: 2 = 2.251-5.700 kg: 3 = 5.701-27.000 kg: 4 = 27.001-272.000 kg: 5 = acima de 272.000 kg

Mas quando envolvemos muitas variáveis ​​como essa, o segredo é isolar para apenas um valor (como a massa de cada aeronave) por vez, depois observar outra variável nesse contexto e, finalmente, outra. Abaixo, criaremos um diagrama de violino que nos ajuda a visualizar a distribuição de colisões com pássaros e quando colisões com motores ocorreram ou não. Eles também podem conter diagramas de caixa de bigodes.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

plt.figure(figsize=(12, 6))
sns.violinplot(x='AC_MASS', y='HEIGHT', data=df, hue='ENGINE_STRIKE_IND', palette='Blues', legend=True)
plt.show()


Certo, então nossa hipótese não revelou nada muito interessante, pelo menos desse ângulo. Parece que comparar a massa da aeronave com a altura e as distribuições de colisões relacionadas ao motor e não relacionadas ao motor não resultou em nada muito interessante. As distribuições parecem praticamente as mesmas e as colisões ocorrem com mais frequência em solo.

Vamos voltar nossa atenção para os custos. Qual é o custo dos reparos ajustado pela inflação, para cada massa de aeronave, tanto para colisões relacionadas ao motor quanto para colisões não relacionadas ao motor?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

plt.figure(figsize=(12, 6))

sns.violinplot(x='AC_MASS', 
               y='COST_REPAIRS_INFL_ADJ', data=df, 
               hue='ENGINE_STRIKE_IND', 
               palette='Blues', 
               inner='quartile', 
               legend=True)
plt.show()


Embora nossos gráficos de violino sejam um pouco tênues devido aos poucos registros que documentam custos, podemos observar que aeronaves com massa de 4,0 (27.001-272.000 kg) são particularmente propensas a custos mais altos para colisões relacionadas ao motor. Em geral, colisões relacionadas ao motor têm maior probabilidade de apresentar custos maiores para valores de `AC_MASS` de 3 ou mais.

Vamos também contabilizar "outros" custos relacionados à perda de receita, hotéis para passageiros devido a voos parados, custo de descarte de combustível e outros custos não relacionados a danos para o operador.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

plt.figure(figsize=(12, 6))
df["TOTAL_COST"] = df["COST_REPAIRS_INFL_ADJ"] + df["COST_OTHER_INFL_ADJ"]

sns.violinplot(x='AC_MASS', 
               y='TOTAL_COST', data=df, 
               hue='ENGINE_STRIKE_IND', 
               palette='Blues', 
               inner='quartile', 
               legend=True)
plt.show()


Vemos aumentos crescentes nos custos relacionados a impactos com motores quando a massa é 4, e isso aumenta significativamente no nível 5. Isso faz sentido, pois aeronaves grandes transportam muitos passageiros que são prejudicados, e provavelmente também possuem os motores mais caros para substituir. Também é interessante notar que aeronaves enormes têm custos quase insignificantes relacionados a impactos que não envolvem os motores.

## Exercício

Será que `SPEED` pode ser uma variável determinante para `COST_REPAIRS_INFL_ADJ`? Faça uma análise abaixo e apresente sua conclusão.

In [ ]:
# COLOQUE O CÓDIGO AQUI, USE MAIS CÉLULAS SE NECESSÁRIO


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

Não há correlação entre a velocidade e o custo dos reparos nos registros de colisões com pássaros. O gráfico de dispersão e o coeficiente de correlação não indicam relação.

In [ ]:
df.plot.scatter(x="SPEED",y="COST_REPAIRS_INFL_ADJ")

In [ ]:
df[["SPEED","COST_REPAIRS_INFL_ADJ"]].corr(method='pearson')